In [1]:
import whisper
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
model = whisper.load_model("medium", device = device)

c:\Users\famel1x\Desktop\MS_cutter\.venv\Lib\site-packages\whisper\__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=de

In [2]:
print(device)
result = model.transcribe("clip_13.mp3")
result["text"]

cuda


' Я не знаю, даже что сказать, как прокомментировать. Почему вообще у тебя возникают люди, желающие тебя убить? Так они у всех популярных личностей возникают. У меня больше. Обычно фанаты сумасшедшие, безумные, возникают у личностей культовых именно. Ну да, согласен, их больше. Но мне кажется, в любом случае, у абсолютно любого чела, который уже сильно популярный, у него все равно будут какие-то отбитые хейтеры. Это же неизбежно. Представь, у тебя будет на ролике 100 просмотров. Я представляю, каждый человек будет ставить лайк либо дизлайк. Даже если у тебя будет 100 зрителей, у тебя будет, очевидно, хотя бы два или три дизлайка.'

In [ ]:
import os
import whisper
from moviepy.editor import VideoFileClip
from tqdm import tqdm
device = "cuda" if torch.cuda.is_available() else "cpu"

def extract_audio_from_video(video_path, output_audio_path):
    """
    Извлекает аудио из видеофайла и сохраняет его как отдельный аудиофайл.

    :param video_path: Путь к видеофайлу.
    :param output_audio_path: Путь для сохранения аудиофайла.
    """
    video = VideoFileClip(video_path)
    audio = video.audio
    audio.write_audiofile(output_audio_path)
    video.close()

def transcribe_audio_with_whisper(audio_path, model):
    """
    Распознает текст из аудиофайла с использованием модели Whisper.

    :param audio_path: Путь к аудиофайлу.
    :param model: Загруженная модель Whisper.
    :return: Распознанный текст.
    """
    result = model.transcribe(audio_path)
    return result['text']

def process_videos(videos_dir, output_dir, model_name="base"):
    """
    Обрабатывает все видеофайлы в директории, извлекает аудио и распознает текст.

    :param videos_dir: Директория с видеофайлами.
    :param output_dir: Директория для сохранения субтитров.
    :param model_name: Название модели Whisper для распознавания речи (tiny, base, small, medium, large).
    """
    # Проверка на наличие выходной директории
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Загружаем модель Whisper
    print(f"Загружается модель Whisper: {model_name}")
    model = whisper.load_model(model_name, device=device)

    video_files = [f for f in os.listdir(videos_dir) if f.endswith(('.mp4', '.avi', '.mov', '.mkv'))]

    for video_file in tqdm(video_files, desc="Processing videos"):
        video_path = os.path.join(videos_dir, video_file)
        audio_path = os.path.splitext(video_path)[0] + ".mp3"
        output_text_path = os.path.join(output_dir, os.path.splitext(video_file)[0] + ".txt")

        # Извлекаем аудио из видео
        extract_audio_from_video(video_path, audio_path)

        # Распознаем текст с использованием Whisper
        print(f"Распознавание текста для {video_file}...")
        transcription = transcribe_audio_with_whisper(audio_path, model)

        # Сохраняем текст в файл
        with open(output_text_path, 'w', encoding='utf-8') as f:
            f.write(transcription)

        

if __name__ == "__main__":
    # Пример использования
    videos_directory = "clips"        # Директория с видеофайлами
    output_directory = "subtitles"    # Директория для сохранения текстов субтитров

    # Обрабатываем видео с использованием модели Whisper (можно выбрать другую модель: tiny, base, small, medium, large)
    process_videos(videos_directory, output_directory, model_name="medium")


In [8]:


import os
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip
from tqdm import tqdm
import re

def split_text_into_sentences(text):
    """
    Разделяет текст на предложения.
    """
    sentences = re.split(r'(?<=[.!?]) +', text)
    return sentences

def clean_text(text):
    """
    Очищает текст от ненужных символов.
    """
    # Удаляем все не-русские и не-английские символы, кроме основных знаков препинания
    clean_text = re.sub(r'[^а-яА-ЯёЁa-zA-Z0-9\s,.!?]', '', text)
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()
    return clean_text

def add_subtitles_to_video(video_path, subtitle_text, output_path, fontsize=24, color='white', bg_color='black', margin=10):
    """
    Добавляет субтитры к видео.

    :param video_path: Путь к исходному видео.
    :param subtitle_text: Текст субтитров.
    :param output_path: Путь для сохранения видео с субтитрами.
    :param fontsize: Размер шрифта субтитров.
    :param color: Цвет текста субтитров.
    :param bg_color: Цвет фона субтитров.
    :param margin: Отступ от нижней части видео.
    """
    # Разделяем текст на предложения
    sentences = split_text_into_sentences(subtitle_text)
    num_sentences = len(sentences)
    
    # Загружаем видео
    video = VideoFileClip(video_path)
    duration = video.duration

    if num_sentences == 0:
        print(f"No subtitles found for {video_path}")
        return

    # Рассчитываем длительность каждого субтитра
    subtitle_duration = duration / num_sentences

    # Создаем текстовые клипы для субтитров
    subtitle_clips = []
    for i, sentence in enumerate(sentences):
        start = i * subtitle_duration
        end = (i + 1) * subtitle_duration

        txt_clip = (TextClip(sentence, fontsize=fontsize, color=color, font="Arial", method='caption', size=(video.w*0.8, None))
                    .set_position(('center', video.h - fontsize - margin))
                    .set_start(start)
                    .set_duration(subtitle_duration)
                    .on_color(color=bg_color, col_opacity=0.6, padding=10))
        
        subtitle_clips.append(txt_clip)
    
    # Объединяем видео и субтитры
    final = CompositeVideoClip([video, *subtitle_clips])

    # Сохраняем видео с субтитрами
    final.write_videofile(output_path, codec="libx264", audio_codec="aac")

def process_videos_with_subtitles(videos_dir, subtitles_dir, output_dir):
    """
    Обрабатывает все видео в директории, добавляя к ним субтитры.

    :param videos_dir: Директория с видео.
    :param subtitles_dir: Директория с текстовыми файлами субтитров (с тем же именем, но .txt).
    :param output_dir: Директория для сохранения видео с субтитрами.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    video_files = [f for f in os.listdir(videos_dir) if f.endswith(('.mp4', '.avi', '.mov', '.mkv'))]
    
    for video_file in tqdm(video_files, desc="Adding subtitles to videos"):
        video_path = os.path.join(videos_dir, video_file)
        subtitle_file = os.path.splitext(video_file)[0] + '.txt'
        subtitle_path = os.path.join(subtitles_dir, subtitle_file)
        
        if not os.path.exists(subtitle_path):
            print(f"Subtitle file not found for {video_file}, skipping.")
            continue
        
        with open(subtitle_path, 'r', encoding='utf-8') as f:
            subtitle_text = f.read()
        
        output_path = os.path.join(output_dir, f"subtitled_{video_file}")
        add_subtitles_to_video(video_path, subtitle_text, output_path)

if __name__ == "__main__":
    # Пример использования
    videos_directory = "clips"            # Директория с видео файлами
    subtitles_directory = "subtitles"      # Директория с текстовыми файлами субтитров
    output_directory = "clips_with_subtitles"  # Директория для сохранения видео с субтитрами

    process_videos_with_subtitles(videos_directory, subtitles_directory, output_directory)


Adding subtitles to videos:   0%|          | 0/58 [00:00<?, ?it/s]


OSError: MoviePy Error: creation of None failed because of the following error:

[WinError 2] Не удается найти указанный файл.

.This error can be due to the fact that ImageMagick is not installed on your computer, or (for Windows users) that you didn't specify the path to the ImageMagick binary in file conf.py, or that the path you specified is incorrect